In [ ]:
from pyspark.sql.types import IntegerType

In [ ]:
housePath = "data/Houses.txt"
consumptionPath = "data/DailyPowerConsumption.txt"
outputPath1 = "outPart1SQL/"
outputPath2 = "outPart2SQL/"

In [ ]:
# Define the dataframes associated with the input files
houseDF = spark.read.format("csv")\
            .option("header", False).option("inferSchema", True)\
            .load(housePath)\
            .withColumnRenamed("_c0", "HouseID")\
            .withColumnRenamed("_c1", "City")\
            .withColumnRenamed("_c2", "Country")\
            .withColumnRenamed("_c3", "SizeSQM")

# Register the temporary table houses 
houseDF.createOrReplaceTempView("houses")

In [ ]:
consumptionDF = spark.read.format("csv")\
                .option("header", False).option("inferSchema", True)\
                .load(consumptionPath)\
                .withColumnRenamed("_c0", "HouseID")\
                .withColumnRenamed("_c1", "Date")\
                .withColumnRenamed("_c2", "kWh")

# Register the temporary table consumption 
consumptionDF.createOrReplaceTempView("consumption")

In [ ]:
# Part 1

# Register a function that given a date in the format "" returns the year  
spark.udf.register("YEAR", lambda date: int(date.split("/")[0]), IntegerType())

In [ ]:
# Consider only the readings associated with the year 2022,
# compute the average daily consumption for each house in the year 2022,
# and select the houses with a daily consumption >30
housesHighConsumptionDF = spark.sql("SELECT HouseID "
                                    + "FROM consumption "
                                    + "WHERE YEAR(Date)=2022 "
                                    + "GROUP BY HouseID "
                                    + "HAVING SUM(kWh)/365>30")

# Register the temporary table housesHighCons 
housesHighConsumptionDF.createOrReplaceTempView("housesHighCons")

In [ ]:
# Select the countries that are never associated with houses with a high average daily
# consumption in the year 2022
res1DF = spark.sql("SELECT DISTINCT Country "
                   + "FROM houses "
                   + "WHERE Country NOT IN ("
                   + " SELECT Country "
                   + " FROM housesHighCons, houses "
                   + " WHERE housesHighCons.HouseID=houses.HouseID)")

In [ ]:
res1DF.write\
    .format("csv")\
    .option("header", False)\
    .save(outputPath1)

In [ ]:
# Part 2
# keep only the houses for which the total power consumption over 2021 is > 10000 kWh
highTotalPowerCons2021DF = spark.sql("SELECT HouseID "
                                     + "FROM consumption "
                                     + "WHERE YEAR(Date)=2021 "
                                     + "GROUP BY HouseID "
                                     + "HAVING SUM(kWh)>10000")

# Register the temporary table housesHighCons2021 
highTotalPowerCons2021DF.createOrReplaceTempView("housesHighCons2021")

In [ ]:
# Select for each country the cities with at least 500 houses with high annual power consumption
coutryCityManyHighConsHousesDF = spark\
            .sql("SELECT City, Country "
                 + "FROM houses, housesHighCons2021 "
                 + "WHERE houses.HouseID=housesHighCons2021.HouseID "
                 + "GROUP BY City, Country "
                 + "HAVING COUNT(*)>500")

# Register the temporary table citiesWithManyHighConsHouses 
coutryCityManyHighConsHousesDF.createOrReplaceTempView("citiesWithManyHighConsHouses")

In [ ]:
# Compute for each country the number of cities with at least 500 houses with high 
# annual power consumption
# We must consider also the countries without cities with at least 500 houses 
# with high annual power consumption
highPwrConsCitiesPerCountryDF = spark\
.sql("SELECT Country, SUM(CityWithManyHighConsHouses) "
     + "FROM ( (SELECT Country, 1 as CityWithManyHighConsHouses FROM citiesWithManyHighConsHouses) "
     +"        UNION ALL "  # Note SQL command "UNION ALL" and not the SQL command "UNION"
     +"        (SELECT Country, 0 as CityWithManyHighConsHouses FROM Houses) "
     +"      ) "
     + "GROUP BY Country")

In [ ]:
highPwrConsCitiesPerCountryDF\
    .write\
    .format("csv")\
    .option("header", False)\
    .save(outputPath2)